IRIS经典数据分类训练,来自[https://www.tensorflow.org/get_started/estimator?hl=zh-cn](https://www.tensorflow.org/get_started/estimator?hl=zh-cn)

In [1]:
import tensorflow as tf
import numpy as np
import requests
import os

In [12]:
# 准备数据

IRIS_TRAINING = "data/iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "data/iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

resp = requests.get(IRIS_TRAINING_URL)
with open(IRIS_TRAINING, 'wb') as f:
    f.write(resp.content)

resp = requests.get(IRIS_TEST_URL)
with open(IRIS_TEST, 'wb') as f:
    f.write(resp.content)


In [13]:
# 加载数据

training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TRAINING,
    target_dtype=np.int,
    features_dtype=np.float32)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TEST,
    target_dtype=np.int,
    features_dtype=np.float32)


In [14]:
# 属性数据

feature_columns = [tf.feature_column.numeric_column("x",shape=[4])]

# 一个3层的分别为10,20,10个神经元的DNN网络,最后有3类
classifier = tf.estimator.DNNClassifier(hidden_units=[10,20,10],
	feature_columns=feature_columns,model_dir="/tmp/iris_model",n_classes=3)




INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/tmp/iris_model', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [15]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": np.array(training_set.data)},
                                                    y=np.array(training_set.target),
                                                    num_epochs=None,
                                                    shuffle=True)


In [16]:
# 开始训练

classifier.train(train_input_fn,steps=2000)


INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Saving checkpoints for 1 into /tmp/iris_model/model.ckpt.


INFO:tensorflow:loss = 309.791, step = 1


INFO:tensorflow:global_step/sec: 459.223


INFO:tensorflow:loss = 13.3048, step = 101 (0.220 sec)


...

INFO:tensorflow:loss = 7.03625, step = 1801 (0.116 sec)


INFO:tensorflow:global_step/sec: 890.157


INFO:tensorflow:loss = 5.62664, step = 1901 (0.111 sec)


INFO:tensorflow:Saving checkpoints for 2000 into /tmp/iris_model/model.ckpt.


INFO:tensorflow:Loss for final step: 12.2087.


In [17]:
# 评估结果

test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(test_set.data)},
    y=np.array(test_set.target),
    num_epochs=1,
    shuffle=False)

accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}\n".format(accuracy_score))

INFO:tensorflow:Starting evaluation at 2017-12-08-02:54:20


INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-2000


INFO:tensorflow:Finished evaluation at 2017-12-08-02:54:20


INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.966667, average_loss = 0.0702967, global_step = 2000, loss = 2.1089



Test Accuracy: 0.966667



In [18]:
# 预测

new_samples = np.array(
    [[6.4, 3.2, 4.5, 1.5],
     [5.8, 3.1, 5.0, 1.7]], dtype=np.float32)
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": new_samples},
    num_epochs=1,
    shuffle=False)

predictions = list(classifier.predict(input_fn=predict_input_fn))
predicted_classes = [p["classes"] for p in predictions]

print(
    "New Samples, Class Predictions:    {}\n"
    .format(predicted_classes))


INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-2000


New Samples, Class Predictions:    [array([b'1'], dtype=object), array([b'2'], dtype=object)]

